In [1]:
from astropy.time import Time
import numpy as np
import pandas as pd
import astropy.units as u

## Load data

In [4]:
fink_grb_data_path = "/user/julien.peloton/fink_mm/"

In [5]:
df = spark.read\
.option("basePath", fink_grb_data_path)\
.format("parquet")\
.load([fink_grb_data_path + "gcn_storage/raw"]).toPandas()
df = df.sort_values("triggerTimejd")

## Simple stats

In [6]:
last_day = df["triggerTimeUTC"].values[-1]
print("first gcn date: {}\nlast gcn date: {}".format(
    df["triggerTimeUTC"].values[0], 
    last_day
))

first gcn date: 2023-05-29T20:15:00.746000000
last gcn date: 2023-09-06T16:02:12.940000000


#### GCN of the last day

In [7]:
df[df["triggerTimeUTC"] == last_day]

,observatory,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day
762,Fermi,GBM,,ivo://nasa.gsfc.gcn/Fermi#GBM_Flt_Pos_2023-09-...,715701737,162.85,12.1,418.998,2023-09-06 18:02:51.713222,2.460194e+06,2023-09-06 16:02:12.940,"<VOEvent xmlns:xsi=""http://www.w3.org/2001/XML...",2023,9,6


### Nb gcn / day

In [8]:
df["dayjd"] = df["triggerTimejd"].round()
res_gb = df.groupby("dayjd").count()["observatory"]
"{:.3f} ± {:.3f} gcn/day".format(res_gb.mean(), res_gb.std())

'12.200 ± 5.189 gcn/day'

### Nb gcn / observatory

In [9]:
df.groupby("observatory").count()

,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day,dayjd
observatory,,,,,,,,,,,,,,,
Fermi,199,199,199,199,199,199,199,199,199,199,199,199,199,199,199
ICECUBE,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
INTEGRAL,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75
LVK,719,719,719,719,719,719,719,719,719,719,719,719,719,719,719
SWIFT,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35


### Nb gcn / day / observatory

In [10]:
observatories = df["observatory"].unique()

In [11]:
for obs in observatories:
    nb_day = []
    for day in df["dayjd"]:
        nb_day.append(len(df[(df["dayjd"] == day) & (df["observatory"] == obs)]))
    
    print("{}: {:.3f} ± {:.3f} gcn/day".format(obs, np.mean(nb_day), np.std(nb_day)))
    print()

LVK: 9.859 ± 4.012 gcn/day

Fermi: 2.639 ± 2.274 gcn/day

INTEGRAL: 1.219 ± 2.567 gcn/day

SWIFT: 0.542 ± 1.220 gcn/day

ICECUBE: 0.122 ± 0.379 gcn/day



### Nb gcn / instruments

In [12]:
for obs in observatories:
    df_tmp = df[df["observatory"] == obs]
    print("--- {} ---".format(obs))
    gb_col = "instrument"
    if obs == "ICECUBE":
        gb_col = "event"
    print(df_tmp.groupby(gb_col).count()["triggerId"].to_markdown())
    print("\n")

--- LVK ---
| instrument          |   triggerId |
|:--------------------|------------:|
| H1                  |           3 |
| H1_L1               |         712 |
| L1                  |           3 |
| MbtaL-Hon-clustered |           1 |


--- Fermi ---
| instrument   |   triggerId |
|:-------------|------------:|
| GBM          |         199 |


--- INTEGRAL ---
| instrument   |   triggerId |
|:-------------|------------:|
| Refined      |           5 |
| Wakeup       |           7 |
| Weak         |          63 |


--- SWIFT ---
| instrument   |   triggerId |
|:-------------|------------:|
| BAT          |          25 |
| FOM          |           1 |
| XRT          |           9 |


--- ICECUBE ---
| event   |   triggerId |
|:--------|------------:|
| BRONZE  |           3 |
| Cascade |           2 |
| GOLD    |           4 |




### GCN latency

Warning: this is the latency between the trigger time and the last gcn emitted for each events. This is not the latency computed with the first gcn emitted just after the emission.

In [13]:
df["jdAckTime"] = Time(df["ackTime"].values).jd
df["latency"] = (df["jdAckTime"] - df["triggerTimejd"]) * 24

#### GCN latency global

In [14]:
"{:.3f} ± {:.3f} hours".format(df["latency"].mean(), df["latency"].std())

'8.058 ± 36.133 hours'

In [15]:
df.groupby("observatory").agg(
    latency_mean=("latency", np.mean),
    latency_std=("latency", np.std),
    latency_min=("latency", np.min),
    latency_max=("latency", np.max)
)

,latency_mean,latency_std,latency_min,latency_max
observatory,,,,
Fermi,5.603533,6.809253,4.000984,58.956907
ICECUBE,11.766766,15.419728,4.098092,52.459303
INTEGRAL,4.328403,2.815960,4.001112,28.390164
LVK,9.260323,43.153304,4.021713,911.909466
SWIFT,4.360859,0.387529,4.007949,6.219362


### Error area

#### error area global

In [16]:
err_squaredeg = (df["err_arcmin"].values * u.arcmin**2).to_value(u.deg**2)
"{:.3f} ± {:.3f} deg²".format(err_squaredeg.mean(), err_squaredeg.std())

'3495.563 ± 4557.938 deg²'

#### error area / observatory

In [17]:
df["err_squaredeg"] = err_squaredeg
df.groupby("observatory").agg(
    err_area_mean=("err_squaredeg", np.mean),
    err_area_std=("err_squaredeg", np.std),
    err_area_min=("err_squaredeg", np.min),
    err_area_max=("err_squaredeg", np.max)
)

,err_area_mean,err_area_std,err_area_min,err_area_max
observatory,,,,
Fermi,0.301486,0.236433,1.666667e-02,0.833333
ICECUBE,0.019113,0.036863,8.330556e-05,0.093102
INTEGRAL,0.000016,0.000001,1.272222e-05,0.000019
LVK,5041.500133,4711.732225,4.364327e+01,30093.712943
SWIFT,0.000004,0.000002,1.388889e-07,0.000005
